In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pickle as pk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.stem.snowball import SnowballStemmer
import nltk
import pickle
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
#from glove import Corpus,glove
import gensim
from gensim import corpora, models
from collections import Counter
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
stemmer = SnowballStemmer('english')
news = pd.read_csv('news_articles.csv')
news.head()
tokenizer = ToktokTokenizer()
stop_words = set(stopwords.words('english'))
ntopics = 8

In [2]:
glove_file = '/home/linu/glove.6B.50d.txt'

In [115]:
news = pd.read_csv('news_articles.csv')
news = news[['Article_Id','Title','Content']].dropna()
titles = news['Title']
contents = news["Content"].tolist()

In [4]:
contents[0]

'At least 14 people died and 17 others were injured after a bus travelling from Hyderabad to Kakinada plunged into a canal from a bridge on the accident-prone stretch of the Hyderabad-Khammam highway in Telangana early Monday morning \r\nThe injured were admitted to the Government General Hospital for treatment \r\n\r\n\r\nSeven people died on the spot and the others succumbed to injuries while undergoing treatment at the hospital  The passengers belonged to the East and West Godavari districts of Andhra Pradesh \r\nThe bus  owned by private operator Yatra Genie  commenced its journey from Hyderabad at 11 30 p m  on Sunday  Khammam Superintendent of Police Shah Nawaz Khan was quoted by the Hindustan Times as saying \r\nThe accident happened around 2 30 a m  when the driver slammed the brakes to avoid a collision with another vehicle coming from the opposite direction on a bridge over Nagarjunsagar project left canal at Nayankangudem village in Khammam district  the daily reported  The 

In [5]:
def clean_doc(doc):
    
    document = re.sub('[^\w_\s-]', ' ',doc) #remove punctuation marks and other symbols
    tokens = nltk.word_tokenize(doc)
    cleaned_corpus = ' '.join([i for i in tokens])
    return cleaned_corpus

In [6]:
cleaned_articles = list(map(clean_doc,contents))

In [7]:
cleaned_articles[0]

'At least 14 people died and 17 others were injured after a bus travelling from Hyderabad to Kakinada plunged into a canal from a bridge on the accident-prone stretch of the Hyderabad-Khammam highway in Telangana early Monday morning The injured were admitted to the Government General Hospital for treatment Seven people died on the spot and the others succumbed to injuries while undergoing treatment at the hospital The passengers belonged to the East and West Godavari districts of Andhra Pradesh The bus owned by private operator Yatra Genie commenced its journey from Hyderabad at 11 30 p m on Sunday Khammam Superintendent of Police Shah Nawaz Khan was quoted by the Hindustan Times as saying The accident happened around 2 30 a m when the driver slammed the brakes to avoid a collision with another vehicle coming from the opposite direction on a bridge over Nagarjunsagar project left canal at Nayankangudem village in Khammam district the daily reported The bus hit the parapet wall of the 

In [8]:
#stop_words

In [9]:
article_vocab = { }

article_vocab = enumerate(cleaned_articles)  #enumerating articles vovcab gives us  keys and indexes of article_vocab

In [10]:
total_words = []

for i in range(0,len(cleaned_articles)):
        tokens = nltk.word_tokenize(cleaned_articles[i])
        
        for w in tokens :
            total_words.append(w)
counts = Counter(total_words)

In [11]:
counts  # we used counts function to see how many times a word is repeated but here we used it enumerating it later

Counter({'At': 298,
         'least': 487,
         '14': 314,
         'people': 1470,
         'died': 178,
         'and': 28785,
         '17': 226,
         'others': 446,
         'were': 2904,
         'injured': 313,
         'after': 2439,
         'a': 22807,
         'bus': 132,
         'travelling': 70,
         'from': 5971,
         'Hyderabad': 122,
         'to': 31904,
         'Kakinada': 2,
         'plunged': 44,
         'into': 1070,
         'canal': 4,
         'bridge': 33,
         'on': 13150,
         'the': 73977,
         'accident-prone': 2,
         'stretch': 11,
         'of': 28389,
         'Hyderabad-Khammam': 1,
         'highway': 38,
         'in': 28799,
         'Telangana': 52,
         'early': 274,
         'Monday': 581,
         'morning': 164,
         'The': 13149,
         'admitted': 71,
         'Government': 74,
         'General': 141,
         'Hospital': 82,
         'for': 10630,
         'treatment': 64,
         'Seven': 7,
  

In [12]:
vocab = {j: i for i,j in enumerate(counts.keys())} #here we create a vocabulary of our docs or of the words in our docs

In [13]:
final_vocab = [i for i in vocab.keys() if i not in stop_words] #removing stop words to lighten the matrix 

In [14]:
final_vocab_dict =  {j: i for i,j in enumerate(final_vocab)}  #enumerating after we remove the stop words

In [15]:
len(final_vocab_dict)

56575

In [16]:
#final_vocab = {i for i in enumerate(final_vocab)}

In [17]:
#giving self vocab from counts into tfidf matrix to get the only the words which are needed 

tfmat = TfidfVectorizer(min_df=1,vocabulary=final_vocab_dict)

In [18]:
tfidf_matrix = tfmat.fit_transform(cleaned_articles)

In [19]:
tfidf_matrix.shape    #4831 docs and 56575 words after removing docs

(4831, 56575)

In [20]:
df = pd.read_csv('glove.6B.50d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in df.T.items()}

In [21]:
glove_Vocab = glove.keys()   #fetching keys from glove 

In [22]:
glove_values = glove.values()  #fetching values from glove

In [23]:
len(glove_values)

399998

In [24]:
final_vocab_list = [i.lower() for i in final_vocab_dict.keys()]  #lowering everything in the words fetched

In [25]:
len(final_vocab_list)

56575

In [26]:
final_dict = []

for i in final_vocab_list:
        if i in list(glove):
            final_dict.append(list(glove[i]))
        else :
            final_dict.append([0]*50)

In [28]:
tfidf_matrix_sparse = csr_matrix(tfidf_matrix) #we put tfidfmatrix into a sparse matrix

In [32]:
tfidf_matrix_sparse.shape

(4831, 56575)

In [37]:
# we put values in sparse matrix so that it processes fast and only return the location and value of non zero element
sparse_matrix = csr_matrix(final_dict)

In [38]:
sparse_matrix.shape

(56575, 50)

In [42]:
tf_dot_gloveval = np.dot(tfidf_matrix_sparse,sparse_matrix) 

In [43]:
tf_dot_gloveval.shape  #returns 4831 docs from tfidfmatrix and 50 dimension values from glove

(4831, 50)

In [63]:
kmeans = KMeans(n_clusters=8,random_state=0,max_iter=4000) #we cluster it all to recommend different all of news

In [64]:
kmeans.fit(tf_dot_gloveval)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=4000,
    n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [105]:
labels = kmeans.labels_

In [106]:
ids = np.array(news['Article_Id'])

In [117]:
recommended = [ ]

for i in range(8):
    
    recommended.append(titles[labels==i][:3])

In [118]:
recommended

[2    US  South Korea begin joint military drill ami...
 3    Illegal construction in Bengaluru  Will my hou...
 7    Dialogue crucial in finding permanent solution...
 Name: Title, dtype: object,
 55     Theri  star Vijay invites Nivin Pauly  apprec...
 71    Pawan Kalyan s Sardaar Gabbar Singh teaser  3 ...
 78    Rajini Murugan star Sivakarthikeyan sings prai...
 Name: Title, dtype: object,
 17    Mumbai-Goa bridge collapse  What we know so fa...
 30    Box office collection   Jil Jung Juk    Deadpo...
 33         14 killed  over 30 injured in China bus fire
 Name: Title, dtype: object,
 6      Infosys shares likely to fall on Tuesday after...
 69      Sardaar Gabbar Singh  pre-release business  P...
 159    Nannaku Prematho  Soggade Chinni Nayana may no...
 Name: Title, dtype: object,
 0    14 dead after bus falls into canal in Telangan...
 1    Pratibha Tiwari molested on busy road   Saath ...
 4    Punjab Gau Rakshak Dal chief held for assaulti...
 Name: Title, dtype: object,
 85